In [7]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.memory import ConversationSummaryBufferMemory
from langchain.schema.runnable import RunnablePassthrough


llm = ChatOpenAI(
  temperature=0.1,
)

memory = ConversationSummaryBufferMemory(
  llm=llm,
  max_token_limit=120,
  return_messages=True
)

examples = [
  {
    "question": "탑건",
    "answer": "🛩️🧑‍✈️🔥"
  },
  {
    "question": "대부",
    "answer": "🧑‍🧑‍🧒🔫🍝"
  },
  {
    "question": "괴물",
    "answer": "🦖🏢👣"
  },
  {
    "question": "미션임파서블",
    "answer": "🕶️💥🔒"
  }
]

example_prompt = ChatPromptTemplate.from_messages([
  ("human", "{question}"),
  ("ai", "{answer}")
])

example_prompt = FewShotChatMessagePromptTemplate(
  example_prompt=example_prompt,
  examples=examples,
)

final_prompt = ChatPromptTemplate.from_messages([
  example_prompt,
  ("system", "You are a emoji master, if question is movie title that you know, you should must using three emoji"),
  MessagesPlaceholder(variable_name="history"),
  ("human", "{question}")
])

def load_memory(input):
  return memory.load_memory_variables({})["history"] # 메모리에 저장되어 있는 내용을 불러옴

chain = RunnablePassthrough.assign(history=load_memory) | final_prompt | llm

def invoke_chain(question):
  result = chain.invoke({"question": question})
  memory.save_context({"input": question}, {"output": result.content}) # 메모리에 저장
  print(result)

In [9]:
invoke_chain("파묘")

content='👻🔍🕵️\u200d♂️'


In [13]:
invoke_chain("슈렉")

content='👹👸🐉'


In [10]:
invoke_chain("겨울왕국")

content='❄️👸⛄'


In [15]:
load_memory({})

[SystemMessage(content='The human says "파묘" and the AI responds with emojis of a ghost, magnifying glass, and detective. The human mentions "겨울왕국" and the AI responds with emojis of snowflake, princess, and snowman. The human asks what movie was just mentioned.'),
 AIMessage(content='미안해요, 방금 질문한 영화는 "겨울왕국"이었습니다.'),
 HumanMessage(content='슈렉'),
 AIMessage(content='👹👸🐉'),
 HumanMessage(content='방금 질문한 영화가 뭐였지?'),
 AIMessage(content='방금 질문한 영화는 "슈렉"이었습니다.')]

In [14]:
invoke_chain("방금 질문한 영화가 뭐였지?")

content='방금 질문한 영화는 "슈렉"이었습니다.'
